<a href="https://colab.research.google.com/github/alisaboori-araz/colab/blob/master/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install --upgrade Pillow

In [ ]:
!pip install Pillow python-barcode

In [ ]:
import csv
from PIL import Image, ImageDraw, ImageFont
from barcode import Code128
from barcode.writer import ImageWriter
import io


# import sys

# def trace(frame, event, arg):
#     print("%s, %s:%d" % (event, frame.f_code.co_filename, frame.f_lineno))
#     return trace


# sys.settrace(trace)


def create_barcode_image(csv_file, index, template_image_path,total_width):
    # Read the CSV file and find the matching row
    with open(csv_file, 'r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip the header row
        for row in csv_reader:
                if int(row[0]) < int(index):
                  continue
                print(f"Index: {index}")
                hex_value = row[1]
                print(row)
                print(hex_value)



                # Generate the barcode

                barcode_writer = ImageWriter()
                barcode_obj = Code128(hex_value, writer=barcode_writer)
                options = {
                'font_size': 7,
                'text_distance': 4,'module_height':12,
                'font_path':"/content/temp/fonts/RobotoMono-Bold.ttf",
                'write_text': False
                }

                if total_width:
                       # Calculate module_width based on desired total_width
                       num_modules = sum(len(i) for i in barcode_obj.build())
                       module_width = total_width / num_modules
                       options['module_width'] = module_width

                barcode_image = barcode_obj.render(writer_options=options)

                # Convert the barcode image to RGBA mode
                barcode_image = barcode_image.convert('RGBA')



                # Open the template image
                template_image = Image.open(template_image_path)

                # Resize barcode text
                barcode_text_x = (template_image.width ) / 2


                # Add text
                id_font = ImageFont.truetype("/content/temp/fonts/RobotoMono-Bold.ttf", size=48, encoding="unic")
                barcode_font = ImageFont.truetype("/content/temp/fonts/RobotoMono-Bold.ttf", size=28, encoding="unic")

                # draw content
                draw = ImageDraw.Draw(template_image)
                template_image.paste(barcode_image, (200, 115), barcode_image)
                draw.text((barcode_text_x, 270), hex_value, fill="black", font=barcode_font, align="center")
                draw.text((220, 360), str(index), fill="black", font=id_font )


                # Save the result
                template_image.save(f'{index}.png')
                print(f'Image saved as {f"{index}.png"}')
                index=str(int(index)+1)
                # return

        print(f"Index {index} not found in the CSV file.")



# Example usage
csv_file = '/content/csv-1601-2000.txt'
index = input("Enter the index number: ")
template_image = '/content/template_image.png'
# output_image = f"output_image_{index}.png"

create_barcode_image(csv_file, index, template_image, 58)






In [ ]:
import os
import zipfile

def zip_files_with_prefix(prefix, zip_filename):
    # Get the current working directory
    current_dir = os.getcwd()

    # List all files in the current directory
    files_to_zip = [f for f in os.listdir(current_dir) if f.endswith(prefix) and not(f.startswith("template_image"))]

    # Create a ZipFile object
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for file in files_to_zip:

            # new_filename = f'new_{original_filename}'

            # Rename the file before zipping
            # os.rename(original_filename, new_filename)

            zipf.write(file)
            os.remove(file)
            print(f'File {file} added to the zip. and removed from directory')

    print(f'All files starting with "{prefix}" have been zipped into {zip_filename}')

# Example usage
zip_files_with_prefix('png', 'images.zip')
